In [152]:
import itertools
import time

start_time = time.perf_counter()

input_file = open("input.txt")
input = input_file.read().strip().split('\n')
input_file.close()

cubes = {}

# PART 1
# coord_count = 3
# max_neighbor_count = 26

# PART 2
coord_count = 4
max_neighbor_count = 80

def get_neighbor_cubes(coords):
    adjustments = list(itertools.product([1, -1, 0], repeat=coord_count))
    
    neighbors = []
    
    for adj in adjustments:
        if (all(x==0 for x in adj)):
            continue;
        
        new_coords = list(map(lambda x, y: x + y, coords, adj))
        key = '|'.join(map(str, new_coords))
        
        if key in cubes:
            neighbors.append(cubes[key]);
        
    return neighbors

def get_missing_expansion_coords(coords):
    adjustments = list(itertools.product([1, -1, 0], repeat=coord_count))
    
    missing_for_coords = []
    
    for adj in adjustments:
        new_coords = list(map(lambda x, y: x + y, coords, adj))
        key = '|'.join(map(str, new_coords))
        
        if not key in cubes:
            missing_for_coords.append(tuple(map(lambda x, y: x + y, coords, adj)))
        
    return missing_for_coords

def connect_cube_neighbors():
    for coords, cube in cubes.items():
        if not cube.has_all_neighbors:
            cube.set_neighbors(get_neighbor_cubes(cube.coords))

def expand_cube_grid():
    global cubes
    cubes_to_add = {}
    
    for coords, cube in cubes.items():
        if not cube.has_all_neighbors:
            missing_coords = get_missing_expansion_coords(cube.coords)
            for coords in missing_coords:
                cube = Cube(coords, False)
                
                key = '|'.join(map(str, coords))
                cubes_to_add[key] = cube
                
    cubes = {**cubes, **cubes_to_add}

def update_cube_states():
    for coords, cube in cubes.items():
        cube.determine_upcoming_state()

    for coords, cube in cubes.items():
        cube.apply_upcoming_state()
                
class Cube:
    def __init__(self, coords, is_active):
        self.coords = coords
        self.is_active = is_active
        self.upcoming_state = None
        
        self.neighbors = []
        self.has_all_neighbors = False
    
    def set_neighbors(self, neighbors):
        self.neighbors = neighbors
        
        if (len(self.neighbors) == max_neighbor_count):
            self.has_all_neighbors = True
    
    def determine_upcoming_state(self):
        active_neighbors = filter(lambda neighbor: (neighbor.is_active), self.neighbors)
        active_neighbor_count = len(list(active_neighbors))
        
        self.upcoming_state = self.is_active
        
        if not self.is_active and active_neighbor_count == 3:
            self.upcoming_state = True
            
        if self.is_active and not active_neighbor_count in [2, 3]:
            self.upcoming_state = False
    
    def apply_upcoming_state(self):
        self.is_active = self.upcoming_state
    
    def __repr__(self):
        return str(self.is_active)
        
for y, row in enumerate(input):
    for x, cell in enumerate(list(row)):
        coords = [x, y] + [0]*(coord_count-2)
        key = '|'.join(map(str, coords))
        
        cube = Cube(tuple(coords), cell == '#')
        cubes[key] = cube

for cycle in range(0, 6):
    expand_cube_grid()
    connect_cube_neighbors()
    update_cube_states()
    
active_cubes = list(filter(lambda cube: (cube.is_active), cubes.values()))

print(len(active_cubes))

end_time = time.perf_counter()
print(end_time - start_time, 'seconds')

2532
33.227688099999796 seconds
